In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%matplotlib inline
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,SpatialDropout1D
from tensorflow.keras.layers import LSTM,Dropout
from keras.layers import Bidirectional
from tensorflow.keras.optimizers import RMSprop,Adam

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("/kaggle/input/twitter-sentiment-analysis-hatred-speech/train.csv")
test=pd.read_csv("/kaggle/input/twitter-sentiment-analysis-hatred-speech/test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
def drop(te):
    te.drop("id",axis=1,inplace=True)

In [ ]:
drop(train)

In [ ]:
drop(test)

In [ ]:
train.isnull().any()

Lets check if their is any null value

In [ ]:
test.isnull().any()

In [ ]:
train["label"].value_counts()

In [ ]:
sns.countplot("label",data=train)

Data is imbalanced we will balanced it later before sending it to our Model

# DATA PREPROCESSING

lEt initiliaze our stemmer and tokenizer and do some data processing

In [ ]:
tweettoken = TweetTokenizer(strip_handles=True, reduce_len=True)

In [ ]:
stemmer=PorterStemmer()

In [ ]:
collect=[]
collecttest=[]
def preprocess(t,kpc):
    tee=re.sub('[^a-zA-Z]'," ",t)
    tee=tee.lower()
    res=tweettoken.tokenize(tee)
    for i in res:
        if i in stopwords.words('english'):
            res.remove(i)
    rest=[]
    for k in res:
        rest.append(stemmer.stem(k))
    ret=" ".join(rest)
    if kpc==1:
        collect.append(ret)
    elif kpc==0:
        collecttest.append(ret)

In [ ]:
def splitpro(t,q,m):
         for j in range(q):
                 preprocess(t["tweet"].iloc[j],m)

In [ ]:
splitpro(train,31962,1)

In [ ]:
splitpro(test,17197,0)

In [ ]:
len(collect)

In [ ]:
len(collecttest)

In [ ]:
len(test)

In [ ]:
collect[:5]

In [ ]:
collecttest[:5]

In [ ]:
val=train["label"].values

In [ ]:
val

# DATA BALANCING AND GOING FORWARD WITH ML ALGORITHMS

In [ ]:
def bow(ll):
    cv=CountVectorizer(max_features=200)
    x=cv.fit_transform(ll).toarray()
    return x
    

In [ ]:
y=bow(collect)

In [ ]:
y[0]

In [ ]:
len(y[0][:])

Balancing imbalanced dataset

In [ ]:
from imblearn.under_sampling import NearMiss

In [ ]:
tt=NearMiss()
x_us,y_us=tt.fit_sample(y,val)

In [ ]:
x_us.shape

In [ ]:
(x_train,x_test,y_train,y_test) = train_test_split(x_us,y_us, train_size=0.80, random_state=42)

In [ ]:
x_train

In [ ]:
rnd_clf=RandomForestClassifier(n_estimators=200,random_state=42)

In [ ]:
rnd_clf.fit(x_train,y_train)

In [ ]:
rnd_clf.score(x_test,y_test)

In [ ]:
mm=[300,400,500,600]
for i in mm:
    rnd_clf=RandomForestClassifier(n_estimators=i,random_state=42)
    rnd_clf.fit(x_train,y_train)
    t=rnd_clf.score(x_test,y_test)
    print(t)
    print("*"*40)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train,y_train)
clf.score(x_train,y_train)

In [ ]:
params={
    "eta":[0.01,0.2],
    "min_child_weight":[1,2,3,4,5,6,7,8,9,10],
    "max_depth":[3,6,8,10,15,20,25,30],
    "gamma":[0.0,0.1,0.2,0.3,0.4,0.5,0.6],
    "subsample":[0.5,0.6,0.7,0.8,0.9],
    "colsample_bytree":[0.6,0.7,0.8,0.8],
    "reg_alpha":[0,0.001,0.005,0.01,0.05],
    "learning_rate":[0.05,0.10,0.15,0.20,0.25,0.30],
    "max_leaf_nodes":[8,16,24,32,40],
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
classifier=xgboost.XGBClassifier()

In [ ]:
random=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring="roc_auc",cv=5,verbose=3)

In [ ]:
random.fit(x_train,y_train)

In [ ]:
random.best_estimator_

In [ ]:
classifier=xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, eta=0.01, gamma=0.1,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=25,
              max_leaf_nodes=40, min_child_weight=1,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=0.05, reg_lambda=1,
              scale_pos_weight=1, subsample=0.6, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [ ]:
classifier.fit(x_train,y_train)
classifier.score(x_test,y_test)

In [ ]:
from sklearn import svm
C = [1,10,20,25,30,35,40,50]

In [ ]:
for i in C:
    svc = svm.SVC(kernel='linear', C=i)
    svc.fit(x_train,y_train)
    t=svc.score(x_test,y_test)
    print(t)

In [ ]:
for i in C:
    svc = svm.SVC(kernel='rbf', C=i)
    svc.fit(x_train,y_train)
    t=svc.score(x_test,y_test)
    print(t)

# GOING FORWARD WITH KERAS

In [ ]:
oneh=[]
oneht=[]
def hot(cc,k):
    for i in cc:
        if k==1:
            oneh.append(one_hot(i,10000))
        elif k==0:
            oneht.append(one_hot(i,10000))

In [ ]:
hot(collect,1)

In [ ]:
hot(collecttest,0)

In [ ]:
len(oneh[0])

In [ ]:
oneh[:1]

In [ ]:
len(oneh)

In [ ]:
len(oneht)

In [ ]:
max=0
for i in oneh:
    tq=len(i)
    if tq>max:
        max=tq
print(max)

In [ ]:
sent=40
emoneh=pad_sequences(oneh,padding="pre",maxlen=sent)
emoneht=pad_sequences(oneht,padding="pre",maxlen=sent)

In [ ]:
emoneh[:1]

In [ ]:
emoneht[:1]

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(emoneh,val,train_size=0.80,random_state=42)

In [ ]:
model=Sequential()
model.add(Embedding(10000,100,input_length=sent))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(1, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=50,batch_size=300,verbose=2)

In [ ]:
ytest

In [ ]:
val

In [ ]:
model.predict(emoneht)